In [1]:
import syft as sy
import numpy as np
from sqlalchemy import create_engine
from syft.core.node.common.node_table import Base
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey
from syft.core.adp.scalar.gamma_scalar import GammaScalar
from syft.core.tensor.autodp.initial_gamma import InitialGammaTensor
from syft.core.tensor.autodp.intermediate_gamma import IntermediateGammaTensor
from syft.core.adp.entity import DataSubject
from syft.core.adp.entity import DataSubjectGroup
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from syft.core.io.virtual import create_virtual_connection
from syft.core.adp.vm_private_scalar_manager import VirtualMachinePrivateScalarManager

In [2]:
# Data
child = np.array([1,2,3,4], dtype=np.int32)

In [3]:
# Entities
traskmaster = DataSubject(name="Andrew")
amber = DataSubject(name="Amber")
puppy = DataSubject(name="Their puppy whose name I forgot")
DSG = DataSubjectGroup([traskmaster, amber])

SM = VirtualMachinePrivateScalarManager()

entity_trask = np.array([traskmaster, traskmaster, traskmaster, traskmaster], dtype=object)
entity_amber = np.array([amber, amber, amber, amber], dtype=object)
entity_puppies = np.array([puppy, puppy, puppy, puppy])
dsg_ent = np.array([DSG, DSG, DSG, DSG], dtype=object)

In [4]:
# Various InitialGammaTensors to play with
tensor_trask = InitialGammaTensor(values=child, min_vals=np.zeros_like(child), max_vals=np.ones_like(child) * 5, entities=entity_trask, scalar_manager=SM)
tensor_amber = InitialGammaTensor(values=child, min_vals=np.zeros_like(child), max_vals=np.ones_like(child)* 5, entities=entity_amber, scalar_manager=SM)
tensor_dsg = InitialGammaTensor(values=child * 2, min_vals=np.zeros_like(child), max_vals=np.ones_like(child) * 10, entities=dsg_ent, scalar_manager=SM)
tensor_puppy = InitialGammaTensor(values=child * 3, min_vals = np.zeros_like(child), max_vals=np.ones_like(child) * 15, entities=entity_puppies, scalar_manager=SM) 

In [5]:
# Create our reference tensor using Andrew's and Amber's tensors
reference_tensor = tensor_trask + tensor_amber
assert isinstance(reference_tensor, IntermediateGammaTensor)
print(reference_tensor._entities())

[DSG['<DataSubject:Andrew>', '<DataSubject:Amber>']
 DSG['<DataSubject:Andrew>', '<DataSubject:Amber>']
 DSG['<DataSubject:Andrew>', '<DataSubject:Amber>']
 DSG['<DataSubject:Andrew>', '<DataSubject:Amber>']]


In [6]:
# Test to see if this works for 2D arrays too
child_2d = np.random.randint(low=1, high=5, size=(3, 3), dtype=np.int32)
entities_2d = np.array([[traskmaster, traskmaster, traskmaster], [traskmaster, traskmaster, traskmaster], [traskmaster, traskmaster, traskmaster]], dtype=object)
tensor_2d = InitialGammaTensor(values=child_2d, min_vals=np.zeros_like(child_2d), max_vals=np.ones_like(child_2d) * 5, entities=entities_2d, scalar_manager=SM)
assert tensor_2d.shape == child_2d.shape

IGT_2D = tensor_2d + tensor_2d
assert IGT_2D.shape == IGT_2D._values().shape
assert IGT_2D.shape == IGT_2D._entities().shape

In [7]:
# Use the reference tensor to check private_private equality, starting with gt
comparison_tensor = tensor_puppy + tensor_puppy
assert isinstance(comparison_tensor, IntermediateGammaTensor)
assert comparison_tensor.shape == reference_tensor.shape

comparison_result = reference_tensor > comparison_tensor

In [8]:
# Now let's see if we were right.
print(reference_tensor._values())
print(comparison_tensor._values())

[2. 4. 6. 8.]
[ 6. 12. 18. 24.]


In [9]:
print(comparison_result._values())

[0. 0. 0. 0.]


In [10]:
comparison_result._entities()

array([DSG['<DataSubject:Andrew>', '<DataSubject:Amber>', '<DataSubject:Their puppy whose name I forgot>'],
       DSG['<DataSubject:Andrew>', '<DataSubject:Amber>', '<DataSubject:Their puppy whose name I forgot>'],
       DSG['<DataSubject:Andrew>', '<DataSubject:Amber>', '<DataSubject:Their puppy whose name I forgot>'],
       DSG['<DataSubject:Andrew>', '<DataSubject:Amber>', '<DataSubject:Their puppy whose name I forgot>']],
      dtype=object)

In [11]:
lt_comparison_result = reference_tensor < comparison_tensor
print(lt_comparison_result._values())

[1. 1. 1. 1.]


In [12]:
eq_result = reference_tensor == comparison_tensor

In [13]:
eq_result._values()

array([0., 0., 0., 0.])

In [21]:
eq_result2 = comparison_tensor == (reference_tensor * 3)
eq_result2._values()

array([1., 1., 1., 1.])

In [19]:
(comparison_tensor*3)._values()

array([18., 36., 54., 72.])